<a href="https://colab.research.google.com/github/ShounakDas101/AIML_Hari/blob/main/ElectronPhotonClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import h5py
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision import transforms
import torch.optim as optim

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

# Specify the path to the file in your Google Drive
file_path = '/content/drive/MyDrive/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
CHECKPOINT_PATH = '/content/drive/MyDrive/CHECKPOINTS'

# Check if the file exists
if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
else:
    print(f"The file '{file_path}' does not exist.")

The file '/content/drive/MyDrive/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5' exists.


Use for Dell Gaming Laptop

In [ ]:
'''
# clearing cuda cache memory
import gc
torch.cuda.empty_cache()
gc.collect()
'''

In [ ]:
import h5py

def find_keys(hdf5_obj, path='/'):
    """Recursively find keys in an HDF5 file."""
    keys = []
    for key in hdf5_obj[path].keys():
        full_path = f"{path}/{key}"
        keys.append(full_path)
        if isinstance(hdf5_obj[full_path], h5py.Group):
            keys.extend(find_keys(hdf5_obj, full_path))
    return keys

# Open the HDF5 file
file_path = '/content/drive/MyDrive/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'  # Replace with your actual file path
with h5py.File(file_path, 'r') as hdf5_file:
    # Find keys starting from the root
    all_keys = find_keys(hdf5_file)

# Print the found keys
for key in all_keys:
    print(key)


//X
//y


In [4]:
# import dataset

# importing electron dataset and seperating images and labels
electron_dataset = h5py.File("/content/drive/MyDrive/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5","r")
electron_imgs=np.array(electron_dataset["X"])
electron_labels=np.array(electron_dataset["y"],dtype=np.int64)

# importing photon dataset and seperating images and labels
photon_dataset = h5py.File("/content/drive/MyDrive/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5","r")
photon_imgs=np.array(photon_dataset["X"])
photon_labels=np.array(photon_dataset["y"],dtype=np.int64)

In [5]:
print(electron_imgs.shape)
print(electron_labels)
print(photon_imgs.shape)
print(photon_labels)

(249000, 32, 32, 2)
[1 1 1 ... 1 1 1]
(249000, 32, 32, 2)
[0 0 0 ... 0 0 0]


Display Purpose ONLY

In [ ]:
import matplotlib.pyplot as plt
print(electron_imgs.shape)
for electron_imgs1 in electron_imgs[:3]:
  # Display the first image from electron_imgs
  plt.imshow(electron_imgs1[:, :, 0])  # Assuming it's a 2-channel image; adjust if needed
  plt.title('Electron Image ch1')
  plt.show()

  # Display the first image from photon_imgs
  plt.imshow(electron_imgs1[:, :, 1])  # Assuming it's a 2-channel image; adjust if needed
  plt.title('Electron Image ch2')
  plt.show()



In [ ]:
import matplotlib.pyplot as plt
print(electron_imgs.shape)
for photon_imgs1 in photon_imgs[:3]:
  # Display the first image from electron_imgs
  plt.imshow(photon_imgs1[:, :, 0])  # Assuming it's a 2-channel image; adjust if needed
  plt.title('Electron Image ch1')
  plt.show()

  # Display the first image from photon_imgs
  plt.imshow(photon_imgs1[:, :, 1])  # Assuming it's a 2-channel image; adjust if needed
  plt.title('Electron Image ch2')
  plt.show()



In [7]:
# concatenate electron and photon images/labels
img_arrs = torch.Tensor(np.vstack((photon_imgs,electron_imgs)))
labels = torch.Tensor(np.hstack((photon_labels,electron_labels))).to(torch.int64)
img_arrs = img_arrs.permute(0,3,1,2)
print(img_arrs.shape)

torch.Size([498000, 2, 32, 32])


In [8]:
del electron_imgs,photon_imgs,electron_labels,photon_labels

In [9]:
from torch.utils.data import Dataset, DataLoader, random_split

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {'image': self.data[idx], 'label': self.labels[idx]}
        return sample

In [10]:
# Assuming img_arrs and labels are already defined
custom_dataset = CustomDataset(img_arrs, labels)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(custom_dataset))
valid_size = int(0.2 * len(custom_dataset))
test_size = len(custom_dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(custom_dataset, [train_size, valid_size, test_size])
print(len(valid_dataset)+len(test_dataset)+len(train_dataset))

498000


In [11]:
# Create DataLoaders for training, validation, and test sets
batch_size = 1024
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [12]:
# next(iter(...)) catches the first batch of the data loader
# If shuffle is True, this will return a different batch every time we run this cell
# For iterating over the whole dataset, we can simple use "for batch in data_loader: ..."
data_inputs = next(iter(train_loader))

# The shape of the outputs are [batch_size, d_1,...,d_N] where d_1,...,d_N are the
# dimensions of the data point returned from the dataset class
print("Data inputs", data_inputs['image'].shape, "\n")
print("Full",data_inputs)

Data inputs torch.Size([1024, 2, 32, 32]) 

Full {'image': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
      

In [75]:
my_conv1 = nn.Conv2d(in_channels=2, out_channels=8, kernel_size=3, padding=1)
output=my_conv1((data_inputs['image']))
print(output.shape)
relu = nn.ReLU()
output=output.view(batch_size,-1)
print(output.shape[0],output.shape[1])
fc = nn.Linear(output.shape[1],2)
fc(output)

torch.Size([1024, 8, 32, 32])
1024 8192


tensor([[0.1070, 0.1395],
        [0.1090, 0.0875],
        [0.0823, 0.0947],
        ...,
        [0.0670, 0.1119],
        [0.0769, 0.1195],
        [0.0744, 0.0913]], grad_fn=<AddmmBackward0>)

In [81]:
class MyModel(nn.Module):

    def __init__(self, num_classes=1,dropout_rate=.2):
        super().__init__()
        self.my_conv1 = nn.Conv2d(in_channels=2, out_channels=8, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_rate)

        self.Linear1=nn.Linear(32*32*8,32*32*1)
        self.Linear2=nn.Linear(32*32*1,32*8)
        self.Linear3=nn.Linear(32*8,32)

        self.Linear=nn.Linear(32,num_classes)

    def forward(self, x):
        x['image']=self.my_conv1(x['image'])
        x['image']=self.relu(x['image'])
        x['image'] = self.dropout(x['image'])
        x['image']=x['image'].view(batch_size,-1)

        x['image'] = self.Linear1(x['image'])
        x['image']=self.relu(x['image'])
        x['image'] = self.dropout(x['image'])
        x['image'] = self.Linear2(x['image'])
        x['image']=self.relu(x['image'])
        x['image'] = self.dropout(x['image'])
        x['image'] = self.Linear3(x['image'])
        x['image']=self.relu(x['image'])
        x['image'] = self.dropout(x['image'])
        x['image'] = self.Linear(x['image'])
        return x['image']



In [82]:
model = MyModel(num_classes=1)
model.to(device)
print(model)

MyModel(
  (my_conv1): Conv2d(2, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (Linear1): Linear(in_features=8192, out_features=1024, bias=True)
  (Linear2): Linear(in_features=1024, out_features=256, bias=True)
  (Linear3): Linear(in_features=256, out_features=32, bias=True)
  (Linear): Linear(in_features=32, out_features=1, bias=True)
)


In [ ]:
# i=0

# for batch_idx, batch in enumerate(train_loader):
#     # Process your batch here

#     # Check if the current batch is the last one
#     is_last_batch = batch['image'].shape[0] < batch_size

#     # Print information or perform actions based on whether it's the last batch
#     # print(f"Batch {batch_idx + 1}, Batch Size: {len(batch)}, Is Last Batch: {is_last_batch}")
#     if(is_last_batch):
#       break

#     # Continue processing the batch as needed
#     preds = model(batch)


In [83]:
def eval_model(model, data_loader):
    model.eval() # Set model to eval mode
    true_preds, num_preds = 0., 0.

    with torch.no_grad(): # Deactivate gradients for the following code
        for data_inputs in data_loader:
            is_last_batch = data_inputs['image'].shape[0] < batch_size
            if is_last_batch:
                break

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs['image'] = data_inputs['image'].to(device)
            data_inputs['label'] = data_inputs['label'].to(device)
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1)
            preds = torch.sigmoid(preds) # Sigmoid to map predictions between 0 and 1
            pred_labels = (preds >= 0.5).long() # Binarize predictions to 0 and 1

            # Keep records of predictions for the accuracy metric (true_preds=TP+TN, num_preds=TP+TN+FP+FN)
            true_preds += (pred_labels == data_inputs['label']).sum()
            num_preds += batch_size

    acc = true_preds / num_preds
    print(f"Accuracy of the model: {100.0*acc:4.2f}%")

In [84]:
# Input to the optimizer are the parameters of the model: model.parameters()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

loss_module = nn.BCEWithLogitsLoss()


def train_model(model, optimizer, data_loader, val_loader,loss_module, num_epochs=100):
    train_losses = []  # Changed variable name from train_loss to train_losses
    val_losses = []

    # Set model to train mode
    model.train()
    running_loss = 0.0
    # Training loop
    for epoch in tqdm(range(num_epochs)):
        for data_inputs in data_loader:
            # Check if the current batch is the last one
            is_last_batch = data_inputs['image'].shape[0] < batch_size
            if is_last_batch:
                break

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs['image'] = data_inputs['image'].to(device)
            data_inputs['label'] = data_inputs['label'].to(device)

            ## Step 2: Run the model on the input data
            preds = model(data_inputs)

            # Ensure that the predictions have the same data type as the labels
            preds = preds.squeeze(dim=1).to(data_inputs['label'].float())

            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_inputs['label'].float())

            ## Step 4: Perform backpropagation
            optimizer.zero_grad()
            loss.backward()

            ## Step 5: Update the parameters
            optimizer.step()

            running_loss += loss.item() * batch_size  # batch_size

        train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(train_loss)

        #validation plase
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
          for data_inputs in val_loader:
            is_last_batch = data_inputs['image'].shape[0] < batch_size
            if is_last_batch:
                break

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs['image'] = data_inputs['image'].to(device)
            data_inputs['label'] = data_inputs['label'].to(device)

            ## Step 2: Run the model on the input data
            preds = model(data_inputs)

            # Ensure that the predictions have the same data type as the labels
            preds = preds.squeeze(dim=1).to(data_inputs['label'].float())

            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_inputs['label'].float())
            running_loss += loss.item() * batch_size
        val_loss = running_loss / len(val_loader.dataset)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs} - Train loss {train_loss}, Validation Loss {val_loss}")
        eval_model(model, test_loader)


In [85]:
train_model(model, optimizer, train_loader, val_loader,loss_module)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/100 - Train loss 0.6922939431414273, Validation Loss 0.6911501327575928


  1%|          | 1/100 [00:11<19:37, 11.89s/it]

Accuracy of the model: 49.81%
Epoch 2/100 - Train loss 0.8895621471656716, Validation Loss 0.6909721337743552


  2%|▏         | 2/100 [00:22<18:21, 11.23s/it]

...

Accuracy of the model: 55.58%
Epoch 99/100 - Train loss 0.638965460842393, Validation Loss 1.1485546519574392


 99%|█████████▉| 99/100 [18:25<00:11, 11.04s/it]

Accuracy of the model: 61.57%
Epoch 100/100 - Train loss 0.6477814559805208, Validation Loss 1.1785675355229512


100%|██████████| 100/100 [18:36<00:00, 11.16s/it]

Accuracy of the model: 58.84%


In [36]:
eval_model(model, test_loader)

Accuracy of the model: 63.49%
